In [1]:
import pandas as pd
import numpy as np

### Reading dataframes for each protein into python 
Renaming columns to be easily accessable, normalizing the excitation and emission spectra, taking sum of column 

In [314]:
df_mEmerald = pd.read_csv('mEmerald_fpbase_spectra.csv')
df_mEmerald.fillna(0, inplace = True)
df_mTagBFP2 = pd.read_csv('mTagBFP2_fpbase_spectra.csv')
df_mTagBFP2.fillna(0, inplace = True)
df_mCherry = pd.read_csv('mCherry_fpbase_spectra.csv')
df_mCherry.fillna(0, inplace = True)
df_mNeptune2p5 = pd.read_csv('mNeptune2p5_fbpase_spectra.csv')
df_mNeptune2p5.fillna(0, inplace = True)

mEmerald_columns = df_mEmerald.rename(columns ={'mEmerald ex': 'excitation', 'mEmerald em': 'emission'})
mTagBFP2_columns = df_mTagBFP2.rename(columns ={'mTagBFP2 ex': 'excitation', 'mTagBFP2 em': 'emission'})
mCherry_columns = df_mCherry.rename(columns ={'mCherry ex': 'excitation', 'mCherry em': 'emission'})
mNeptune2p5_columns = df_mNeptune2p5.rename(columns ={'mNeptune2.5 ex': 'excitation', 'mNeptune2.5 em': 'emission'})

spectra_list = [mEmerald_columns, mTagBFP2_columns, mCherry_columns, mNeptune2p5_columns]
for j in range(0,4):
    excitation_column = spectra_list[j]['excitation']
    area = np.trapz(excitation_column, dx = 1)
    normalized = excitation_column/area
    excitation_column = normalized
    spectra_list[j]['excitation'] = normalized
    new_area = np.trapz(excitation_column, dx = 1)
    column = spectra_list[j]['emission']
    area_emission = np.trapz(column, dx = 1)
    normalized_em = column/area_emission
    column = normalized_em
    spectra_list[j]['emission'] = normalized_em
    new_area_em = np.trapz(column, dx = 1)

### Function to solve emission bin area using i and j 
Corresponding lambda_min and lambda_max to the min and max wavelengths in channel i in a list of the 200 total bins 

In [22]:
spectra_list = [mEmerald_columns, mTagBFP2_columns, mCherry_columns, mNeptune2p5_columns]

def calc_emission_bin(i, j):
    bins_list = [*range(400, 900, 10)]
    total_bins = 4*bins_list 
    df_spectra = spectra_list[j]
    lambda_min = total_bins[i]
    lambda_max = total_bins[i+1]
    bin_i = range(lambda_min, lambda_max)
    df_sliced_data = df_spectra.loc[df_spectra['wavelength'].isin(bin_i)]
    # print(df_sliced_data)
    sliced_data_emmision = df_sliced_data['emission']
    emission_bin = np.trapz(sliced_data_emmision)
    print('Emission bin sum for i = ', i, 'and j = ', j, 'is', emission_bin)
    return emission_bin

### Function to retrieve excitation value using j 
Excitation value corresponding to excitation wavelenghts for each protein: 405nm, 488nm, 561nm, 637nm. Found these values on csv files

In [5]:
mEmerald_excitation = [0.1732, 0.9982, 0.0043, 0.0027]
mTag_excitation = [0.9588, 0, 0, 0]
mCherry_excitation = [0.0198, 0.0762, 0.6394, 0.0067]
mNeptune_excitation = [0.0279, 0.0717, 0.5819, 0.1519]

In [18]:
excitation_list = [mEmerald_excitation, mTag_excitation, mCherry_excitation, mNeptune_excitation]
def calc_excitation(i, j): 
     excitation_protein = excitation_list[j]
     if 0 <= i < 50: 
          excitation = excitation_protein[0]
     elif 50 <= i < 100: 
          excitation = excitation_protein[1]
     elif 100 <= i < 150:
          excitation = excitation_protein[2]
     elif 150 <= i < 200:
          excitation = excitation_protein[3]
     else:
          print('Invalid i value')
     print('Excitation value for i = ', i, 'and j = ', j, 'is', excitation)
     return excitation

### Function to index over brightness values using a given j 
Found brightness values on slack 

In [16]:
def calc_brightness(j):
    brightness_list = [39.1, 32.38, 15.85, 22.8]
    brightness = brightness_list[j]
    print('Brightness value for j = ', j, 'is', brightness)
    return brightness

### Function to solve for matrix A just inputting i and j value 
All above functions called in this function to reduce reduncancy in solving for the matrix value

In [221]:
def calc_a(i, j):
    k = calc_k(i, j)
    brightness_value = calc_brightness(j)
    excitation_value = calc_excitation(i, j)
    emission_bin_area = calc_emission_bin(i, j)
    a = k * brightness_value * excitation_value * emission_bin_area
    print('Matrix A for image channel i = ', i, ' and spectrum j = ', j, 'is ', a)
    return a

calc_a(89, 1)

Brightness value for j =  1 is 32.38
Excitation value for i =  89 and j =  1 is 0
Emission bin sum for i =  89 and j =  1 is 0.0
Matrix A for image channel i =  89  and spectrum j =  1 is  0.0


0.0

### Function for the variable $k_i$
We are trying to give each fluorophore a fair chance. Each of our illumination wavelengths is targetted mainly at a specific fluorophore. Experimentally, we can choose the illumination strengths. So, we want to balance out our illumination strengths so that a similar amount of information (photons) is collected from each fluorophore emissions.

Depends on the channel--each set of 50 channels will have a different $k_i$. Could make it depend on the brightnesses.

In [223]:
def calc_a_constant_k(i, j):
    k = calc_k(i, j)
    brightness_value = calc_brightness(j)
    excitation_value = calc_excitation(i, j)
    emission_bin_area = calc_emission_bin(i, j)
    a = k * brightness_value * excitation_value * emission_bin_area
    print('Matrix A for image channel i = ', i, ' and spectrum j = ', j, 'is ', a)
    return a


In [278]:
def calc_k(i, j):
    spectra_list = [mEmerald_columns, mTagBFP2_columns, mCherry_columns, mNeptune2p5_columns]
    illumination_wavelengths = [405, 488, 561, 637]
    brightness_list = [39.1, 32.38, 15.85, 22.8]
    brightness = brightness_list[j]
    method = 'use max aij/make "K" all ones'
    if method == 'small if large b':
        m = 5
        if 0 <= i < 50:
            k = 1 / (.95 * m)
        elif 50 <= i < 100:
            k = 1 / (.94 * m)
        elif 100 <= i < 150:
            k = 1 / (.92 * m)
        elif 150 <= i < 200:
            k = 1 / (.93 * m)
    elif method == 'scale by b':
        k = 1 / brightness
    elif method == 'using max aij/make "K" all ones':
        if 0 <= i < 50:
            k = 1 / calc_a_constant_k(5 ,1)
        elif 50 <= i < 100:
            k = 1 / calc_a_constant_k(61 ,0)
        elif 100 <= i < 150:
            k = 1 / calc_a_constant_k(124 ,3)
        elif 150 <= i < 200:
            k = 1 / calc_a_constant_k(174 ,3)
    elif method == 'using good k value':
        if 0 <= i < 50:
            k = 1 / calc_a_constant_k(i ,j)
        elif 50 <= i < 100:
            k = 1 / (.94 * m)
        elif 100 <= i < 150:
            k = 1 / (.92 * m)
        elif 150 <= i < 200:
            k = 1 / (.93 * m)
        # not completed yet, needs replacing of code
    else:
        k = 1
    return k

calc_k(10, 0)

B = np.zeros((199, 4)) ##5 rows (horizontal), 2 columns (vertical)

for j in range(0, 4):
    for i in range(0, 199):
        B[i, j] = calc_k(i, j)

print(B)

[[1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1.

### For loop for matrix A 

In [265]:
A = np.zeros((199, 4)) ##5 rows (horizontal), 2 columns (vertical)

for j in range(0, 4):
    for i in range(0, 199):
        A[i, j] = calc_a(i, j)

print(A)

Brightness value for j =  0 is 39.1
Excitation value for i =  0 and j =  0 is 0.1732
Emission bin sum for i =  0 and j =  0 is 0.0
Matrix A for image channel i =  0  and spectrum j =  0 is  0.0
Brightness value for j =  0 is 39.1
Excitation value for i =  1 and j =  0 is 0.1732
Emission bin sum for i =  1 and j =  0 is 0.0
Matrix A for image channel i =  1  and spectrum j =  0 is  0.0
Brightness value for j =  0 is 39.1
Excitation value for i =  2 and j =  0 is 0.1732
Emission bin sum for i =  2 and j =  0 is 0.0
Matrix A for image channel i =  2  and spectrum j =  0 is  0.0
Brightness value for j =  0 is 39.1
Excitation value for i =  3 and j =  0 is 0.1732
Emission bin sum for i =  3 and j =  0 is 0.0
Matrix A for image channel i =  3  and spectrum j =  0 is  0.0
Brightness value for j =  0 is 39.1
Excitation value for i =  4 and j =  0 is 0.1732
Emission bin sum for i =  4 and j =  0 is 0.0
Matrix A for image channel i =  4  and spectrum j =  0 is  0.0
Brightness value for j =  0 is

In [269]:
np.savetxt("array.csv", A, delimiter=",")

#Save array as csv

Small version of generating $A$ by only creating part of it.

In [268]:
# outer loop
for i in range(1, 11):
    # nested loop
    # to iterate from 1 to 10
    for j in range(1, 11):
        # print multiplication
        print(i * j, end=' ')
    print()

1 2 3 4 5 6 7 8 9 10 
2 4 6 8 10 12 14 16 18 20 
3 6 9 12 15 18 21 24 27 30 
4 8 12 16 20 24 28 32 36 40 
5 10 15 20 25 30 35 40 45 50 
6 12 18 24 30 36 42 48 54 60 
7 14 21 28 35 42 49 56 63 70 
8 16 24 32 40 48 56 64 72 80 
9 18 27 36 45 54 63 72 81 90 
10 20 30 40 50 60 70 80 90 100 


In [256]:
A = np.zeros((5, 2))

for i in range(0, 5):
    # A[i,j]= calc_a(i, j)
    print('looping through i values')
    print('i =', i, ', j =', j)

for j in range(0, 2):
    # A[i,j] = calc_a(i, j)
    print('looping through j values')
    print('i =', i, ', j =', j)
print(A)
np.savetxt("smallA.csv", A, delimiter=",")

looping through i values
i = 0 , j = 1
looping through i values
i = 1 , j = 1
looping through i values
i = 2 , j = 1
looping through i values
i = 3 , j = 1
looping through i values
i = 4 , j = 1
looping through j values
i = 4 , j = 0
looping through j values
i = 4 , j = 1
[[0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]]


In [250]:
os.getcwd()

'/Users/simone/Documents/GitHub/hyperspectral'

### Making sure each spectra dataframs has data for 400-900nm

In [ ]:
spectra_list = [mEmerald_columns, mTagBFP2_columns, mCherry_columns, mNeptune2p5_columns]

In [316]:
wavelength = 701
for index in range(401, 601):
    mEmerald_columns.loc[len(mEmerald_columns.index)] = [wavelength, 0.0, 0.0]
    wavelength = wavelength + 1

mEmerald_columns

,wavelength,excitation,emission
0,300.0,0.001432,0.0
1,301.0,0.001298,0.0
2,302.0,0.001193,0.0
3,303.0,0.001100,0.0
4,304.0,0.001005,0.0
...,...,...,...
596,896.0,0.000000,0.0
597,897.0,0.000000,0.0
598,898.0,0.000000,0.0
599,899.0,0.000000,0.0


In [318]:
wavelength = 651
for index in range(341, 591):
    mTagBFP2_columns.loc[len(mTagBFP2_columns.index)] = [wavelength, 0.0, 0.0]
    wavelength = wavelength + 1

mTagBFP2_columns

,wavelength,excitation,emission
0,310.0,0.000403,0.0
1,311.0,0.000399,0.0
2,312.0,0.000393,0.0
3,313.0,0.000383,0.0
4,314.0,0.000372,0.0
...,...,...,...
586,896.0,0.000000,0.0
587,897.0,0.000000,0.0
588,898.0,0.000000,0.0
589,899.0,0.000000,0.0


In [321]:
wavelength = 851
for index in range(621, 671):
    mNeptune2p5_columns.loc[len(mNeptune2p5_columns.index)] = [wavelength, 0.0, 0.0]
    wavelength = wavelength + 1

mNeptune2p5_columns

,wavelength,excitation,emission
0,230.0,0.000013,0.0
1,231.0,0.000005,0.0
2,232.0,0.000008,0.0
3,233.0,0.000005,0.0
4,234.0,0.000005,0.0
...,...,...,...
666,896.0,0.000000,0.0
667,897.0,0.000000,0.0
668,898.0,0.000000,0.0
669,899.0,0.000000,0.0


### Function to calculate the excitation values from the illumination wavelengths
Figure out how to go into csv and grab the specific excitation values from the dataframe that you found earlier. Can use for loop and/or function. 

In [334]:
# def calc_excitation(): 
# #     for excitation in mEmerald_columns: 
# #         if mEmerald_columns['wavelength'] == 405:
# #             print(mEmerald_columns.loc[illumination_wavelengths, :])

# # calc_excitation()

# # for excitation in mEmerald_columns:
# #     # spectra_list = [mEmerald_columns, mTagBFP2_columns, mCherry_columns, mNeptune2p5_columns]
# #     illumination_wavelengths = [405, 488, 561, 637]

